In [7]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from unicode import join_jamos
from rapidfuzz import process, fuzz

# Google Sheets API 인증 및 접근
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name("key.json", scope)
gc = gspread.authorize(creds)

# 스프레드시트 URL 설정
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1PzFuHuF2DvMPdSI-TK4Kq5LjPwyGzIxH3Q0Drnqft20/edit?usp=sharing"

# URL을 통해 스프레드시트 열기
doc = gc.open_by_url(spreadsheet_url)

# 특정 시트 선택 (사본 페이지)
sheet = doc.worksheet("상병 Category의 사본")

# 시트의 모든 데이터를 가져오기 (2D 리스트 형태로 가져옴)
data = sheet.get_all_values()

# 데이터프레임으로 변환
df = pd.DataFrame(data[1:], columns=data[0])  # 첫 번째 행은 컬럼명으로 설정


class HangulSearch:
    eng_to_kor_dict = {
        "r": "ㄱ",
        "R": "ㄲ",
        "s": "ㄴ",
        "e": "ㄷ",
        "E": "ㄸ",
        "f": "ㄹ",
        "a": "ㅁ",
        "q": "ㅂ",
        "Q": "ㅃ",
        "t": "ㅅ",
        "T": "ㅆ",
        "d": "ㅇ",
        "w": "ㅈ",
        "W": "ㅉ",
        "c": "ㅊ",
        "z": "ㅋ",
        "x": "ㅌ",
        "v": "ㅍ",
        "g": "ㅎ",
        "k": "ㅏ",
        "o": "ㅐ",
        "i": "ㅑ",
        "O": "ㅒ",
        "j": "ㅓ",
        "p": "ㅔ",
        "u": "ㅕ",
        "P": "ㅖ",
        "h": "ㅗ",
        "y": "ㅛ",
        "n": "ㅜ",
        "b": "ㅠ",
        "m": "ㅡ",
        "l": "ㅣ",
    }

    def __init__(self, dataframe):
        self.dataframe = dataframe

    def convert_eng_to_kor(self, eng_text):
        """영어 입력을 한글 자모로 변환하는 함수"""
        return "".join(self.eng_to_kor_dict.get(char, char) for char in eng_text)

    def correct_query(self, query):
        """검색어를 DataFrame의 데이터를 참고하여 보정하는 함수"""
        if all(
            char.isalpha() and char.islower() for char in query
        ):  # 모든 문자가 알파벳 소문자인 경우 한글로 변환
            query = join_jamos(self.convert_eng_to_kor(query))

        max_score = 0
        corrected = query
        for column in self.dataframe.columns:
            closest_match = process.extractOne(
                query, self.dataframe[column], scorer=fuzz.WRatio
            )
            if closest_match and closest_match[1] > max_score:
                max_score = closest_match[1]
                corrected = (
                    closest_match[0] if max_score > 50 else query
                )  # 유사도 50% 이상인 경우만 반환

        return corrected

    def search_in_dataframe(self, query):
        # 입력된 query에 해당하는 모든 열에서 부분일치 검색
        result = self.dataframe[
            self.dataframe.apply(
                lambda row: row.astype(str)
                .str.contains(query, case=False, na=False)
                .any(),
                axis=1,
            )
        ]

        # 부분일치 결과가 없으면 오탈자 보정을 통해 검색
        if result.empty:
            corrected_query = self.correct_query(query)
            result = self.dataframe[
                self.dataframe.apply(
                    lambda row: row.astype(str)
                    .str.contains(corrected_query, case=False, na=False)
                    .any(),
                    axis=1,
                )
            ]
            return result, corrected_query

        return result, None


# 사용자 입력 예시
user_input = "상1병"  # 사용자가 입력한 검색어

# HangulSearch 클래스 인스턴스 생성 및 검색 수행
searcher = HangulSearch(df)
search_results, corrected_query = searcher.search_in_dataframe(user_input)
print(search_results, "-----------------"*100,
      corrected_query)
# 결과 출력
if corrected_query:
    print(
        f"'{user_input}'에 대한 검색 결과가 없습니다. 대신 '{corrected_query}'로 검색한 결과입니다:"
    )
else:
    print(f"'{user_input}'에 대한 검색 결과입니다:")

# search_results

    불완전 상병 코드                   불완전 상병명                         청구 카테고리  \
37      K02.0                 법랑질에 제한된 우식                    청구 기초 및 기본진료   
38      K02.1                      상아질 우식            청구 기초 및 기본진료, 보존, 외과   
39      K02.2                     시멘트질 우식            청구 기초 및 기본진료, 보존, 외과   
40      K02.3                    정지된 치아우식                청구 기초 및 기본진료, 보존   
41      K02.5                치수 노출이 있는 우식          청구 기초 및 기본진료, 보존, 근관치료   
42      K02.8                     기타 치아우식            청구 기초 및 기본진료, 보존, 외과   
44     K03.00                      교합면 마모                    청구 기초 및 기본진료   
45     K03.01                      인접면 마모                    청구 기초 및 기본진료   
46     K03.10   치아의 쐐기결손 NOS (치약마모, 굴곡파절)                청구 기초 및 기본진료, 보존   
47     K03.18  치아의 기타 명시된 마모 (습관, 전통, 직업)                청구 기초 및 기본진료, 보존   
48      K03.2                      치아의 침식                    청구 기초 및 기본진료   
50     K03.80                      민감 상아질                청구 기초 및 기본진료, 보존   
5